In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

## astropy.wcs
Implements the FITS WCS standard and some commonly used distortion conventions.

This tutorial will show how to create a WCS object from a FITS file and how to use it to transform coordinates.

In [2]:
import os
from astropy.io import fits
from astropy import wcs

Open a file with `astropy.fits` and look at it.

In [6]:
sip_file_name = os.path.join('sip.fits')

sip_file = fits.open(sip_file_name)
sip_file.info()

Filename: sip.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      62   ()              


To create a WCS object pass the header with the WCS kyewords to astropy.wcs.WCS (Primary header in this case).

In [7]:
w = wcs.WCS(sip_file[0].header)

To perform the transformation from detector to sky, including distortions, pass x and y and an 'origin'. The third argument, 'origin', indicates whether the coordinates are 1-based (like FITS), or 0-based (like python).

In [8]:
ra, dec = w.all_pix2world([1, 1], [2, 2], 1)
print(ra, dec)

[ 202.39347618  202.39347618] [ 47.1772851  47.1772851]


Perfom the inverse transformation - from sky to detector coordinates.

If analytical inverse is not available (often the case in the presence of distortion), then an iterative inverse is performed.

In [10]:
print(w.all_world2pix(ra, dec, 1))

[array([ 1.00000005,  1.00000005]), array([ 2.00000006,  2.00000006])]


In some cases it is useful to omit the distortion and perform the core WCS transforms only:

In [11]:
ra, dec = w.wcs_pix2world([1, 1], [2, 2], 1)
print(ra, dec)

[ 202.39299121  202.39299121] [ 47.17731548  47.17731548]


In [12]:
w.wcs_world2pix(ra, dec, 1)

[array([ 1.,  1.]), array([ 2.,  2.])]

The WCS object can be changed and the new WCS can be written out as a new header.
`
By default only the primary WCS keywords are written out to the header. Pass a keyword `relax=True` to write out the SIP distortion.

In [13]:
# The original WCS
w.printwcs()

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 202.482322805  47.175118929999996  
CRPIX : 128.0  128.0  
PC1_1 PC1_2  : 0.000249756880272  0.00023017780974399999  
PC2_1 PC2_2  : 0.000230428519265  -0.00024996577057699999  
CDELT : 1.0  1.0  
NAXIS    : 0 0


In [16]:
w.wcs.crpix = [200, 200]
#w.to_header()
w.to_header(relax=True)

WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =                200.0 / Pixel coordinate of reference point            
CRPIX2  =                200.0 / Pixel coordinate of reference point            
PC1_1   =    0.000249756880272 / Coordinate transformation matrix element       
PC1_2   =    0.000230177809744 / Coordinate transformation matrix element       
PC2_1   =    0.000230428519265 / Coordinate transformation matrix element       
PC2_2   =   -0.000249965770577 / Coordinate transformation matrix element       
CDELT1  =                  1.0 / [deg] Coordinate increment at reference point  
CDELT2  =                  1.0 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TAN-SIP'       / TAN (gnomonic) projection + SIP distortions    
CTYPE2  = 'DEC--TAN-SIP'    

Exercise 1:

- Create a WCS object from the file. 

dist_file_name = 'dist_lookup.fits.gz'

This file contains all distortions typical for HST imaging data - SIP, lookup_table and det2im (detector to image - correcting detector irregularities). The lookup table and det2im distortions are stored in separate extensions so you will need to pass as a second argument to `wcs.WCS` the file object (already opened with astropy.io.fits).

- Look at the file object with the `info()` method. The lookup_table and det2im distortions are saved in separate extensions.

- Modify one of the WCS keywords and save it to file. (As some of the distortion is saved in extensions, use the method `to_fits()` to save the entire WCS.

Exercise 2:

Using the same file create a WCS object for the alternate WCS in this header, by passing also `key='O'` to wcs.WCS.
Commpare the two WCSs using the `printwcs()` method`